In [1]:
import pandas as pd
import os
import sys
import fbprophet as fb
from datetime import datetime

In [2]:
import numpy as np
import csv

In [ ]:
file_open = open(r'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\sales_train_validation_10_items.csv')
file_contents = csv.reader(file_open)
file_write = open(r'C:\Users\Rohan\Desktop\Kaggle\M5_Walmart\sales_train_validation_normalized_test1.csv','a',newline='')
print(F"start time is {dt.now()}")
dim_fields = 6
transaction_id = 1
for i,row in enumerate(file_contents):
    start = 6
    if i == 0:
        top_row = row
        normalize = row[:dim_fields]
        normalize.extend(['date','unit_sold','trasaction_id'])
        row1 = normalize
        print(row1)
        row1_final = [row1[0]]
        row1_final.extend(row1[-4:-1])
        row_write = csv.writer(file_write)
        row_write.writerow(row1_final)
        nu_of_fields = len(top_row) - dim_fields
    else:
        start = dim_fields
        for j in range(1,nu_of_fields+1):
            normalize_data = row[:dim_fields]
            #print(j,nu_of_fields,start)
            date = top_row[start]
            #print(date)
            next_trans = row[start]
            normalize_data.extend([date,next_trans,transaction_id])
            #print(normalize_data)
            transaction_id = transaction_id + 1
            start = start + 1
            row2 = normalize_data
            #print(row2)
            row2_final = [row2[0]]
            row2_final.extend(row2[-4:-1])
            row_write.writerow(row2_final)
file_write.close()
print(F"end time is {dt.now()}")

In [3]:
file_path = 'C:\\Users\\Rohan\\Desktop\\Kaggle\\M5_Walmart\\'

In [4]:
df_calendar = pd.read_csv(file_path + 'calendar.csv',parse_dates=[0])
df_calendar[['unwanted','date_sk']] = df_calendar.date_key.str.split("_",expand=True)
df_calendar.drop(columns=['unwanted'],inplace=True)
df_calendar.year = df_calendar.year.astype('int64')
df_calendar.date_sk = df_calendar.date_sk.astype('int64')

In [5]:
df_read_full_file = pd.read_csv(file_path + 'sales_train_validation_normalized.csv')

In [6]:
df_item_store_combo = df_read_full_file[['item_sk','store_sk']][df_read_full_file.duplicated(subset=['item_sk','store_sk']) == False]

In [7]:
df_item_store_combo = df_item_store_combo.reset_index(drop=True)

In [8]:
df_store = pd.read_csv(file_path + 'dim_store.csv')
df_store.set_index(['store_sk'],inplace=True)

In [9]:
df_item_store_combo['store_state'] = df_item_store_combo.store_sk.apply(lambda x: df_store.at[x,'state_id'])

In [10]:
def create_holiday(df_cal,state):      
        if state == 'CA':
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_CA == 1)]
        elif state == 'TX':
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_TX == 1)]
        else:
            df_holiday = df_cal[(df_cal.event_name_1.notna()) | (df_cal.snap_WI == 1)]
        df_holiday['holiday'] = np.where(df_holiday.event_name_1.isna(),'snap',df_holiday.event_name_1)
        df_holiday.rename(columns={'date':'ds'},inplace=True)
        return(df_holiday[['holiday','ds']])

In [11]:
states = []
for i in df_calendar.columns:
    if 'snap' in i.lower():
        states.append(i.split('_')[1])

In [12]:
holiday_ca = create_holiday(df_calendar,states[0])

C:\Users\Rohan\miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Rohan\miniconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
holiday_tx = create_holiday(df_calendar,states[1])

C:\Users\Rohan\miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
holiday_wi = create_holiday(df_calendar,states[2])

C:\Users\Rohan\miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
#df_calendar = df_calendar.set_index(['date'])
def calendar_lookup(x):
    return(df_calendar.at[x,'date_key'])

In [68]:
m_ca = fb.Prophet(holidays=holiday_ca)
m_tx = fb.Prophet(holidays=holiday_tx)
m_wi = fb.Prophet(holidays=holiday_wi)

In [74]:
for index,i in df_item_store_combo.iterrows():
    if index >= 10 and index < 100:
        print(F"loop start time {datetime.strftime(datetime.now(),format='%Y%m%d %H:%M:%S')}")
        print(i['item_sk'],i['store_sk'])
        df_item_sk_1 = df_read_full_file[(df_read_full_file.item_sk == i['item_sk']) & (df_read_full_file.store_sk == i['store_sk'])]
        df_item_sk_1[['unwanted','date_sk2']] = df_item_sk_1.date.str.split("_",expand=True)
        df_item_sk_1.drop(columns=['unwanted','date'],inplace=True)
        df_item_sk_1.rename(columns={'date_sk2':'date_sk'},inplace=True)
        df_item_sk_1.date_sk = df_item_sk_1.date_sk.astype('int64')
        df_item_sk_1 = df_item_sk_1.join(df_calendar.set_index('date_sk'),on=['date_sk'],rsuffix='xxx')
        df1_for_fb = df_item_sk_1[['date','unit_sold']]
        df1_for_fb.rename(columns={'date':'ds','unit_sold':'y'},inplace=True)
        if i['store_state'] == 'CA':
            m = fb.Prophet(holidays=holiday_ca,daily_seasonality=True)
            print("using cali holiday calendar")
        elif i['store_state'] == 'TX':
            m = fb.Prophet(holidays=holiday_tx,daily_seasonality=True)
            print("using texas holiday calendar")
        else:
            m = fb.Prophet(holidays=holiday_wi,daily_seasonality=True)
            print("using wisconsin holiday calendar")
        m.fit(df1_for_fb)
        future = m.make_future_dataframe(periods=28)
        forecast = m.predict(future)
        forecast = forecast[['ds','yhat']][forecast.ds > df1_for_fb.ds.max()]
        df_calendar.set_index(['date'],inplace=True)
        forecast['date_key'] = forecast.ds.apply(lambda x: calendar_lookup(x))
        forecast = forecast[['date_key','yhat']]
        forecast.set_index(['date_key'],inplace=True)
        forecast = forecast.transpose()
        forecast = forecast.iloc[:,-28:]
        forecast.insert(0,'item_sk',i['item_sk'])
        forecast.insert(1,'store_sk',i['store_sk'])
        df_calendar.reset_index(inplace=True)
        forecast.to_csv(file_path + 'walmart_prediction.csv', index=False, header=False, mode='a')
        print(F"loop end time {datetime.strftime(datetime.now(),format='%Y%m%d %H:%M:%S')}")

loop start time 20200622 07:14:52
11 1
using cali holiday calendar
loop end time 20200622 07:15:14
loop start time 20200622 07:15:14
12 1
using cali holiday calendar
loop end time 20200622 07:15:28
loop start time 20200622 07:15:28
13 1
using cali holiday calendar
loop end time 20200622 07:15:40
loop start time 20200622 07:15:40
14 1
using cali holiday calendar
loop end time 20200622 07:15:50
loop start time 20200622 07:15:50
15 1
using cali holiday calendar
loop end time 20200622 07:16:02
loop start time 20200622 07:16:02
16 1
using cali holiday calendar
loop end time 20200622 07:16:12
loop start time 20200622 07:16:12
17 1
using cali holiday calendar
loop end time 20200622 07:16:23
loop start time 20200622 07:16:23
18 1
using cali holiday calendar
loop end time 20200622 07:16:35
loop start time 20200622 07:16:35
19 1
using cali holiday calendar
loop end time 20200622 07:16:49
loop start time 20200622 07:16:49
20 1
using cali holiday calendar
loop end time 20200622 07:16:57
loop start

using cali holiday calendar
loop end time 20200622 07:29:12
loop start time 20200622 07:29:12
95 1
using cali holiday calendar
loop end time 20200622 07:29:19
loop start time 20200622 07:29:19
96 1
using cali holiday calendar
loop end time 20200622 07:29:27
loop start time 20200622 07:29:27
97 1
using cali holiday calendar
loop end time 20200622 07:29:34
loop start time 20200622 07:29:34
98 1
using cali holiday calendar
loop end time 20200622 07:29:40
loop start time 20200622 07:29:40
99 1
using cali holiday calendar
loop end time 20200622 07:29:46
loop start time 20200622 07:29:46
100 1
using cali holiday calendar
loop end time 20200622 07:29:54


In [60]:
df_calendar.reset_index(inplace=True)

In [58]:
forecast.iloc[:,-30:]

date_key,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943
yhat,0.765443,0.840214,0.743319,0.711537,0.761867,0.776413,0.586322,0.707713,0.779334,0.679378,...,0.678629,0.599048,0.674792,0.5803,0.55242,0.608194,0.765814,0.66146,0.589102,0.672126


In [37]:
forecast.head()

date_key,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943
yhat,0.765443,0.840214,0.743319,0.711537,0.761867,0.776413,0.586322,0.707713,0.779334,0.679378,...,0.678629,0.599048,0.674792,0.5803,0.55242,0.608194,0.765814,0.66146,0.589102,0.672126


In [66]:
forecast.to_csv(file_path + 'walmart_prediction.csv', index=False, header=False, mode='a')

In [42]:
forecast1 = forecast[['ds','yhat']]

In [45]:
df_calendar = df_calendar.set_index(['date'])
def calendar_lookup(x):
    return(df_calendar.at[x,'date_key'])

KeyError: "None of ['date'] are in the columns"

In [43]:
forecast1['date_key'] = forecast1.ds.apply(lambda x: calendar_lookup(x))

C:\Users\Rohan\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
forecast1 = forecast1[['date_key','yhat']]

In [45]:
forecast1.set_index(['date_key'],inplace=True)

In [46]:
forecast1 = forecast1.transpose()

In [47]:
forecast1.insert(0,'item_sk',1)

In [48]:
forecast1.insert(1,'store_sk',1)

In [49]:
forecast1.head()

date_key,item_sk,store_sk,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,...,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941,d_1942,d_1943
yhat,1,1,0.762079,0.8222,0.706412,0.696686,0.749047,0.896207,0.772794,0.721267,...,0.675334,0.624892,0.685664,0.571381,0.564116,0.619986,0.771778,0.654178,0.609681,0.676746


In [51]:
forecast1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, yhat to yhat
Data columns (total 32 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   item_sk   1 non-null      int64  
 1   store_sk  1 non-null      int64  
 2   d_1914    1 non-null      float64
 3   d_1915    1 non-null      float64
 4   d_1916    1 non-null      float64
 5   d_1917    1 non-null      float64
 6   d_1918    1 non-null      float64
 7   d_1919    1 non-null      float64
 8   d_1920    1 non-null      float64
 9   d_1921    1 non-null      float64
 10  d_1922    1 non-null      float64
 11  d_1923    1 non-null      float64
 12  d_1924    1 non-null      float64
 13  d_1925    1 non-null      float64
 14  d_1926    1 non-null      float64
 15  d_1927    1 non-null      float64
 16  d_1928    1 non-null      float64
 17  d_1929    1 non-null      float64
 18  d_1930    1 non-null      float64
 19  d_1931    1 non-null      float64
 20  d_1932    1 non-null      float64
 

In [61]:
df_calendar.reset_index(inplace=True)

In [56]:
forecast

,date_key,yhat,item_sk,store_sk
1913,d_1914,0.762079,1,1
1914,d_1915,0.822200,1,1
1915,d_1916,0.706412,1,1
1916,d_1917,0.696686,1,1
1917,d_1918,0.749047,1,1
1918,d_1919,0.896207,1,1
1919,d_1920,0.772794,1,1
1920,d_1921,0.721267,1,1
1921,d_1922,0.780071,1,1
1922,d_1923,0.662868,1,1


In [17]:
df_item_store_combo['store_state'] = df

ValueError: At based indexing on an integer index can only have integer indexers

In [16]:
df_store = pd.read_csv(file_path + 'dim_store.csv')
df_store.set_index(['store_sk'],inplace=True)

In [18]:
df_store.at[1,'state_id']

'CA'

In [19]:
df_item_store_combo.dtypes

index       int64
item_sk     int64
store_sk    int64
dtype: object

In [21]:
df_item_store_combo['store_state'] = df_item_store_combo.store_sk.apply(lambda x: df_store.at[x,'state_id'])

In [34]:
df_item_store_combo

,item_sk,store_sk
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
30485,3045,10
30486,3046,10
30487,3047,10
30488,3048,10


In [46]:
df_calendar

,wm_yr_wk,weekday,wday,month,year,date_key,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,date_sk
date,,,,,,,,,,,,,,
2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,2
2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,3
2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,4
2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1,1965
2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0,1966
2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0,1967


In [47]:
df_item_sk_1

,id,item_sk,store_sk,unit_sold,trasaction_id,date_sk,wm_yr_wk,weekday,wday,month,year,date_key,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,1,1,0,1,1,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_001_CA_1_validation,1,1,0,2,2,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_001_CA_1_validation,1,1,0,3,3,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_001_CA_1_validation,1,1,0,4,4,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,HOBBIES_1_001_CA_1_validation,1,1,0,5,5,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,HOBBIES_1_001_CA_1_validation,1,1,1,1909,1909,11612,Wednesday,5,4,2016,d_1909,NaN,NaN,NaN,NaN,0,0,0
1909,HOBBIES_1_001_CA_1_validation,1,1,3,1910,1910,11612,Thursday,6,4,2016,d_1910,NaN,NaN,NaN,NaN,0,0,0
1910,HOBBIES_1_001_CA_1_validation,1,1,0,1911,1911,11612,Friday,7,4,2016,d_1911,NaN,NaN,NaN,NaN,0,0,0
1911,HOBBIES_1_001_CA_1_validation,1,1,1,1912,1912,11613,Saturday,1,4,2016,d_1912,NaN,NaN,NaN,NaN,0,0,0


In [49]:
df_calendar

,date,wm_yr_wk,weekday,wday,month,year,date_key,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,date_sk
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0,1
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0,2
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0,3
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0,4
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1,1965
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0,1966
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0,1967
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0,1968


In [38]:
states

['CA', 'TX', 'WI']

In [55]:
states[-2:]

['TX', 'WI']